<a href="https://colab.research.google.com/github/brandiegriffin83/brandiegriffin83/blob/main/ITAI2376_L02_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install tensorflow
!pip install matplotlib
!pip install ipywidgets

In [28]:
# Import necessary libraries for VGG16 model and image processing
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image # Used for image handling

# Import for displaying outputs and widgets
from IPython.display import display
import ipywidgets as widgets

# Import for numerical operations
import numpy as np

# Import for working with in-memory data streams
import io

# Introduction
This lab is designed to introduce you to the basics of deep learning by interacting with a pre-built model. You'll understand the workflow of a deep learning project, including data preprocessing, model architecture, and making predictions. The goal is to familiarize yourself with the basics of deep learning without writing any code.


In [29]:
# Load the VGG16 model
model = VGG16(weights='imagenet')

# Display the model architecture
model.summary()


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ fc1 (Dense)                          │ (None, 4096)                │     102,764,544 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 138,357,544 (527.79 MB)

 Trainable params: 138,357,544 (527.79 MB)

 Non-trainable params: 0 (0.00 B)

In [33]:
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
import numpy as np
import ipywidgets as widgets
from IPython.display import display


def load_and_preprocess_image(image_path):
    """
    Loads an image from the specified path, preprocesses it for VGG16,
    and returns the original and processed images.

    Args:
        image_path (str): The path to the image file.

    Returns:
        tuple: A tuple containing the original PIL Image and the
               preprocessed NumPy array.
    """

    if not os.path.exists(image_path):
        raise FileNotFoundError(f"The image file '{image_path}' was not found. "
                                f"Please provide a valid path.")

    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    return img, img_array


# Widget for file upload
uploader = widgets.FileUpload()
display(uploader)

# Output widget to display results
output = widgets.Output()
display(output)

# Function to handle file upload and prediction
def handle_upload(change):
    with output:
        # Clear previous output
        output.clear_output()

        # Get uploaded image data
        uploaded_filename = next(iter(uploader.value))
        content = uploader.value[uploaded_filename]['content']

        # Save the uploaded image to a temporary file
        with open('uploaded_image.jpg', 'wb') as f:
            f.write(content)

        # Process and predict
        try:
            original_img, processed_img = load_and_preprocess_image('uploaded_image.jpg')
            # ... (Rest of your prediction code using processed_img) ...
            print("Image loaded and preprocessed successfully!")

        except FileNotFoundError as e:
            print(e)

# Observe changes in the uploader widget
uploader.observe(handle_upload, names='value')

FileUpload(value={}, description='Upload')

Output()

In [37]:
# Import necessary libraries
from tensorflow.keras.applications.vgg16 import VGG16, decode_predictions, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Load the pre-trained VGG16 model
model = VGG16(weights='imagenet')

# Load and preprocess an image - replace 'path/to/your/image.jpg' with the actual path
img_path = 'path/to/your/image.jpg'

In [38]:
# Upload button to load images
upload = widgets.FileUpload()
display(upload)

# Button to make predictions
predict_button = widgets.Button(description="Make Prediction")
display(predict_button)

# Function to handle button click
def on_click(change):
    if upload.value:  # Check if a file has been uploaded
        img_data = list(upload.value.values())[0]['content']
        img = Image.open(io.BytesIO(img_data))
        img = img.resize((224, 224))

        # Preprocess and predict
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        predictions = model.predict(img_array)
        decoded_predictions = decode_predictions(predictions, top=3)[0]

        # Display predictions
        print(decoded_predictions)
    else:
        print("Please upload an image first.")

predict_button.on_click(on_click)

FileUpload(value={}, description='Upload')

Button(description='Make Prediction', style=ButtonStyle())

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 705ms/step
[('n02085936', 'Maltese_dog', 0.9188059), ('n02098413', 'Lhasa', 0.014036181), ('n02096437', 'Dandie_Dinmont', 0.0133733535)]


# Conclusion and Discussion
Reflect on the lab activities. Discuss how the pre-trained model was able to make predictions, the role of data preprocessing, and the impact of input modifications on the model's predictions.
